In [1]:
from ortools.linear_solver import pywraplp

In [2]:
# Define available resources
resources = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

# Define the intents.
# For have, the first value is the amount to trade and the second is how much the intent holder values that resource.
# These are interpreted as exclusive possible trades; one thing gets traded for one other thing, and only one of the two options.
intents = [
    #intentA
    {'weight': 1.0, 'have': {'A1': (1, 1.0), 'A2': (1, 0.5)}, 
                    'want': {'C1': (1, 1.0), 'B2': (1, 1.0)}},
    #intentB
    {'weight': 1.0, 'have': {'B1': (1, 0.5), 'B2': (1, 1.0)}, 
                    'want': {'C2': (1, 1.0), 'A2': (1, 0.2)}},
    #intentC
    {'weight': 1.0, 'have': {'C1': (1, 1.0), 'C2': (1, 1.0)}, 
                    'want': {'B1': (1, 1.0), 'A1': (1, 1.0)}},
]

In [3]:
# Initialize the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Objective function
objective = solver.Objective()

# Variables to store each intent trade
x = {}
for i, intent in enumerate(intents):
    for have in intent['have']:
        for want in intent['want']:
            x[f"x_{i}_{have}_{want}"] = solver.BoolVar(f"x_{i}_{have}_{want}")

# Conservation laws for each resource
for resource in resources:
    inflow = sum(qty * x[f"x_{i}_{have}_{want}"] for i, intent in enumerate(intents) for have, (qty, _) in intent['have'].items() if have == resource for want in intent['want'])
    outflow = sum(qty * x[f"x_{i}_{have}_{want}"] for i, intent in enumerate(intents) for want, (qty, _) in intent['want'].items() if want == resource for have in intent['have'])
    solver.Add(inflow == outflow)

# One trade per intent
for i, intent in enumerate(intents):
    solver.Add(sum(x[f"x_{i}_{have}_{want}"] for have in intent['have'] for want in intent['want']) <= 1)

# Objective function
objective = solver.Objective()
for i, intent in enumerate(intents):
    for have, (have_qty, have_weight) in intent['have'].items():
        for want, (want_qty, want_weight) in intent['want'].items():
            coefficient = intent['weight'] * (1 / have_weight) * want_weight
            objective.SetCoefficient(x[f"x_{i}_{have}_{want}"], coefficient)

# Set objective function to maximize the weighted boolean variables
objective.SetMaximization()

In [4]:
# Solve the problem
status = solver.Solve()

In [6]:
# Output results
if status == pywraplp.Solver.OPTIMAL:
    print(f"Objective value = {objective.Value()}")
    for i, intent in enumerate(intents):
        for have in intent['have']:
            for want in intent['want']:
                if x[f"x_{i}_{have}_{want}"].solution_value() > 0:
                    print(f"Intent {i + 1} trades {have} for {want}")
else:
    print("The problem does not have an optimal solution.")

Objective value = 3.4000000000000004
Intent 1 trades A2 for C1
Intent 2 trades B1 for A2
Intent 3 trades C1 for B1
